In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
from helperToolz.dicts_and_lists import INT_TO_MONTH

In [2]:
files = sorted(getFilelist('/data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw', '.nc'))
year = 2020
#get a subset of files for that year
yearFiles = [file for file in files if int(file.split('/')[-1].split('_')[-1][0:4]) == year]

# create a maks for germany
mask = makeGermanyMaskforNC('/data/Aldhani/eoagritwin/misc/gadm41_DEU_shp/gadm41_DEU_0.shp', yearFiles[0])

/data/Aldhani/users/potzschf/conda/envs/workhorse/lib/python3.12/site-packages/osgeo/ogr.py:601: FutureWarning: Neither ogr.UseExceptions() nor ogr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [ ]:
tiffPath = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/'
yearCont = []# for collecting number of observations per year

# loop over files and export to .tif at Path locations
for i, file in enumerate(yearFiles):
    
    print(f'start on file {file}')
    
    accDateTimes = getAllDatesS3(file) # possible to take annual subset if entire files list would be passed here
    convertNCtoTIF(file, tiffPath, file.split('/')[-1].split('.')[0] + '.tif', accDateTimes, False, True, LST=False)

start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-01-01.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-01-15.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-02-01.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-02-15.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-03-01.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-03-15.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-04-01.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-04-15.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-05-01.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-05-15.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-06-01.nc
start on file /data/Aldhani/eoagritwin/et/Auxiliary/VZA/raw/Germany_2020-06-15.nc
start on file /d

In [15]:
ncfile = file
storPath = tiffPath
fileName = yearFiles[0].split('/')[-1].split('.')[0] + '.tif'
accDT=accDateTimes
make_uint16 = False
explode = True
LST = False

In [ ]:
for year in [i for i in range(2017,2025,1)]:

    print(f'Start processing .nc files for the year {year}')

    # get a subset of files for that year
    yearFiles = [file for file in files if int(file.split('/')[-1].split('_')[-1][0:4]) == year]

    # create a maks for germany
    mask = makeGermanyMaskforNC('/data/Aldhani/eoagritwin/misc/gadm41_DEU_shp/gadm41_DEU_0.shp', yearFiles[0])

    # set storPath for exported tiffs
    storPath = '/data/Aldhani/eoagritwin/et/Sentinel3/tiffs/'
    LST_path = f'{storPath}LST/daily_observations_all/{year}/'
    Time_path = f'{storPath}Acq_time/{year}/'
    monthly_composites_path = f'{storPath}LST/monthly_composites/{year}/'
    # ensure that directories exist
    [os.makedirs(dir_path, exist_ok=True) for dir_path in [LST_path, Time_path, monthly_composites_path]]

    yearCont = []# for collecting number of observations per year

    # loop over files and export to .tif at Path locations
    for i, file in enumerate(yearFiles):
        
        print(f'start on file {file}')
        
        accDateTimes = getAllDatesS3(file) # possible to take annual subset if entire files list would be passed here
    #     convertNCtoTIF(file, LST_path, file.split('/')[-1].split('.')[0] + '.tif', accDateTimes, False, True)


        dat = getDataFromNC(file)
        monthCont = [] # for collecting number of observations per month
        dailyCont = [] # for collecting number of observations per day
        dailyVals_median = [] # for collection the actual daily LST values (daily median)
        dailyVals_mean = [] # for collection the actual daily LST values (daily mean)
        dailyVals_max = [] # for collection the actual daily LST values (daily mean)
        bnames = []
        df = pd.Series(accDateTimes)
        counts_per_day = df.dt.floor("D").value_counts().sort_index()
        # vectors for indexing over days
        cumulative_day_counts_end = np.asarray(np.cumsum(counts_per_day))
        cumulative_day_counts_start = np.insert(cumulative_day_counts_end, 0 ,0)

        # cumulative_day_counts_start = np.array(cumulative_day_counts_start)
        # cumulative_day_counts_end = np.array(cumulative_day_counts_end)

        # get accquisition times ready for aggregation and export
        time_cube = np.full(dat.shape, None, dtype='float64')
        for i in range(len(df)):
            mask = ~np.isnan(dat[:,:,i])
            time_cube[:,:,i][mask] = df[i].timestamp() # needed conversion as tif export won't work with datetimeobject
        dailyTimeDates_max = []
        ################################################ gives monthly min, max, median composites
        # aggreagate by median
        # stack_list = [
        #     np.nanmedian(dat[:, :, start:end], axis=2)
        #     for start, end in zip(cumulative_day_counts_start[:-1], cumulative_day_counts_end)
        # ] 
        # fin_block = np.dstack(stack_list)

        MM = INT_TO_MONTH[file.rsplit('-', maxsplit=1)[-1].split('.')[0]]
        # bands = [f'{MM}_Day_{b+1}' for b in range(fin_block.shape[2])]
        # fin_block = fin_block * mask[:, :, np.newaxis]
        # fin_block[fin_block == 0] = np.nan
        # exportNCarrayDerivatesComp(file, monthly_composites_path, f'Germany_{year}_{MM}_mean.tif', bands, fin_block)

        # # aggreagate by min
        # stack_list = [
        #     np.nanmin(dat[:, :, start:end], axis=2)
        #     for start, end in zip(cumulative_day_counts_start[:-1], cumulative_day_counts_end)
        # ] 
        # fin_block = np.dstack(stack_list)
        # fin_block = fin_block * mask[:, :, np.newaxis]
        # fin_block[fin_block == 0] = np.nan
        # exportNCarrayDerivatesComp(file, monthly_composites_path, f'Germany_{year}_{MM}_min.tif', bands, fin_block)

        # # aggreagate by max
        # stack_list = [
        #     np.nanmax(dat[:, :, start:end], axis=2)
        #     for start, end in zip(cumulative_day_counts_start[:-1], cumulative_day_counts_end)
        # ] 
        # fin_block = np.dstack(stack_list)
        # fin_block = fin_block * mask[:, :, np.newaxis]
        # fin_block[fin_block == 0] = np.nan
        # exportNCarrayDerivatesComp(file, monthly_composites_path, f'Germany_{year}_{MM}_max.tif', bands, fin_block)

        ########################################################### creates metadata raster
        for l in range(len(counts_per_day)):
            # number of observations per month
            monthCont.append(np.any(~np.isnan(dat[:, :, cumulative_day_counts_start[l]:cumulative_day_counts_end[l]]),axis=2)) # minimum dail obs
            
            # collect the dates to use as bandnames for exported tif stacks
            bnames.append(str(counts_per_day.index[l].date()))

            # collect number of observations per day ( count only one per day!)
            dailyCont.append(np.sum(~np.isnan(dat[:, :, cumulative_day_counts_start[l]:cumulative_day_counts_end[l]]),axis=2))
            
            # # collect actual LST values
            # dailyVals_median.append(np.nanmedian(dat[:, :, cumulative_day_counts_start[l]:cumulative_day_counts_end[l]], axis = 2))
            # dailyVals_mean.append(np.nanmean(dat[:, :, cumulative_day_counts_start[l]:cumulative_day_counts_end[l]], axis = 2))
            dailyVals_max.append(np.nanmax(dat[:, :, cumulative_day_counts_start[l]:cumulative_day_counts_end[l]], axis = 2))

            # collect the acquisition time and dates in timestamp format
            dailyTimeDates_max.append(getValsatMaxIndex(dat[:, :, cumulative_day_counts_start[l]:cumulative_day_counts_end[l]],
                                                        time_cube[:, :, cumulative_day_counts_start[l]:cumulative_day_counts_end[l]]))
   
        # # export daily values
        # exportNCarrayDerivatesInt(file, LST_path, f'Daily_LST_means_{year}_{MM}.tif', bnames, np.dstack(dailyVals_mean), make_uint16=False, numberOfBands=len(dailyVals_mean))
        # exportNCarrayDerivatesInt(file, LST_path, f'Daily_LST_medians_{year}_{MM}.tif', bnames, np.dstack(dailyVals_median), make_uint16=False, numberOfBands=len(dailyVals_median))
        exportNCarrayDerivatesInt(file, LST_path, f'Daily_LST_max_{year}_{MM}.tif', bnames, np.dstack(dailyVals_max), make_uint16=False, numberOfBands=len(dailyVals_max))
        exportNCarrayDerivatesInt(file, Time_path, f'Daily_Time_max_{year}_{MM}.tif', bnames, np.dstack(dailyTimeDates_max), make_uint16=False, numberOfBands=len(dailyTimeDates_max))
        # # export day counts
        # exportNCarrayDerivatesInt(file, storPath + 'Analytics/Count_obs_per_day/', f'Daily_obs_for_{year}_{MM}.tif', bnames, np.dstack(dailyCont), True, numberOfBands=len(dailyCont))
        # # export month counts
        # exportNCarrayDerivatesInt(file, storPath + 'Analytics/Count_obs_per_month/', f'Monthly_Min_DailyObs_{('_').join(file.split('_')[-1].split('-')[:2])}.tif', 'monthly_sum_of_daily_obs', np.nansum(np.dstack((monthCont)), axis = 2), True)
        
        # # collect number of observations per year ( count only one per day!)
        # yearCont.append(np.nansum(np.dstack((monthCont)), axis = 2))

    # # export year counts
    # exportNCarrayDerivatesInt(file, storPath + 'Analytics/Count_obs_per_year/', f'Annual_Min_DailyObs_{file.split('_')[-1].split('-')[0]}.tif', 'annual_sum_of_daily_obs', np.nansum(np.dstack((yearCont)), axis = 2), True)

In [ ]:
def getBluGrnRedBnrFORCEList(filelist):
    '''Takes a list of paths to an exploded FORCE output and returns a list with ordered paths
    First all blue then green, red and bnir bands. Furthermore, paths are chronologically sorted (1,2,3,4..months)'''
    blu = [file for file in filelist if file.split('SEN2H_')[-1].split('_')[0] == 'BLU']
    grn = [file for file in filelist if file.split('SEN2H_')[-1].split('_')[0] == 'GRN']
    red = [file for file in filelist if file.split('SEN2H_')[-1].split('_')[0] == 'RED']
    bnr = [file for file in filelist if file.split('SEN2H_')[-1].split('_')[0] == 'BNR']

    blu = sortListwithOtherlist([int(t.split('-')[-1].split('.')[0]) for t in blu], blu)[-1]
    grn = sortListwithOtherlist([int(t.split('-')[-1].split('.')[0]) for t in grn], grn)[-1]
    red = sortListwithOtherlist([int(t.split('-')[-1].split('.')[0]) for t in red], red)[-1]
    bnr = sortListwithOtherlist([int(t.split('-')[-1].split('.')[0]) for t in bnr], bnr)[-1]

    return sum([blu, grn, red, bnr], [])

In [ ]:
def getExplodedFORCEbyColorAndTime(fileList, colorList, timeList=None):
    """This function help to order paths of FORCEfiles according to a list of color, e.g. ['BLU', 'GRN',...]
    Furthermore, output will be sorted against a list of time, if provided

    Args:
        fileList (list): a list of paths to exploded FORCE output
        colorList (list): a list of the colors to group the output by
    """

color_pattern = rf'_{target_color}_'



In [ ]:
fileList=['/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0068_Y0042/20190101-20191231_060-310_HL_TSA_SEN2L_BLU_TSI_20190630.tif',
 '/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0067_Y0042/20190101-20191231_001-365_HL_TSA_SEN2L_BNR_TSI_20190630.tif',
 '/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0068_Y0042/20190101-20191231_060-310_HL_TSA_SEN2L_BNR_TSI_20190630.tif',
 '/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0067_Y0042/20190101-20191231_001-365_HL_TSA_SEN2L_GRN_TSI_20190630.tif',
 '/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0068_Y0042/20190101-20191231_060-310_HL_TSA_SEN2L_GRN_TSI_20190630.tif',
 '/data/Aldhani/eoagritwin/force/output/Guzinski/1TileTest/X0067_Y0042/20190101-20191231_001-365_HL_TSA_SEN2L_RE1_TSI_20190630.tif']

colorList=['BNR']

for target_color in colorList:
    search_pattern = rf'_{target_color}_'

    print([file for file in fileList if re.search(search_pattern, file)])

In [ ]:
a =random.randint(0,9999)
print(f'{a:04d}')